<a href="https://colab.research.google.com/github/paulcardozo/Parcial2_IA/blob/master/Parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Subir el dataset
Haremos uso del dataset del repositorio de UCI respecto a tiempos de espera y la información para la respuesta de una cola de archivos que serán procesados en un servidor de arquitectura 2D-SOME Bus
https://archive.ics.uci.edu/ml/datasets/Optical+Interconnection+Network+

In [1]:
from google.colab import files
uploaded = files.upload()

Saving optical_interconnection_network.csv to optical_interconnection_network.csv


# Contenido Dataset


*   Node Number: Se refiere a la cantidad de nodos utilizados en el servirdor para lograr el procesamiento del archivo solicitado
*   Thread Number: Es el número de hilos creados para lograr el procesamiento
*   Spatial Distribution: El rendimiento de la red utilizada según los siguientes modelos: Uniform (UN), Hot Region (HR), Bit reverse (BR) and Perfect Shuffle (PS)
*   Temporal Distribution: Se refiere a la generación de los paquetes generados o recibidos por el servidor, estos pueden ser de tipo Client-Server o Asynchronous
*   T/R: Tiempo de transferencia del mensaje medidos en ciclos de cómputo
*   Processor Utilization: Un porcentaje de la cantidad utilizada del procesador
*   Channel Waiting Time: El tiempo de espera de un paquete en el canal de salida que llega al servidor hasta ser ejecutado
*   Input Waiting Time: El tiempo en que un paquete será ejecutado por el servidor
*   Network Response Time: El tiempo que el paquete se encuentra dentro de la red del servidor
*   Channel Utilization: El porcentaje del tamaño del canal que fue utilizado 


# Creación de Pipeline
Se realiza la creación de un pipeline, enfocado en el método de StandardScaler para el preprocesamiento, que da la escala apropiada para las variables discretas, y para el proceso de análisis se realizará DecisionTreeClassifier, para lograr procesar todas las variables necesarias

In [10]:
import io
import pandas as pd
dataset = pd.read_csv(io.BytesIO(uploaded['optical_interconnection_network.csv']))
# El dataset estará cargado en el framework de pandas
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.pipeline import make_pipeline

cleanup_data = {"Thread Number":{4:0,8:1},
        "Spatial Distribution":{"UN":0,"HR":1,"BR":2,"PS":3},
        "Temporal Distribution":{"Client-Server":0,"Asynchronous":1}}
dataset.replace(cleanup_data, inplace=True)

#split dataset in features and target variable
feature_cols = ['T/R', 'Temporal Distribution','Processor Utilization', 'Channel Waiting Time', 'Input Waiting Time','Network Response Time','Channel Utilization','Node Number']
X = dataset[feature_cols] # Features
y = dataset["Thread Number"] # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

classifier = make_pipeline(StandardScaler(),clf)
# Train Decision Tree Classifer

classifier.fit(X_train,y_train)
print(X_test)
print()
y_pred = classifier.predict(X_test)
print(y_pred)
# Model Accuracy, how often is the classifier correct?
print("Precisión del Modelo:",metrics.accuracy_score(y_test, y_pred))


     T/R  Temporal Distribution  ...  Channel Utilization  Node Number
132  0.3                      0  ...             0.713385           64
622  0.3                      1  ...             0.808437           16
525  0.6                      1  ...             0.709583           16
289  1.0                      1  ...             0.872309           64
82   0.3                      0  ...             0.549167           64
..   ...                    ...  ...                  ...          ...
375  0.6                      0  ...             0.786875           16
76   0.7                      0  ...             0.774557           64
286  0.7                      1  ...             0.824323           64
347  0.8                      0  ...             0.932778           16
186  0.7                      1  ...             0.934853           64

[192 rows x 8 columns]

[ 6  1  0  0  0  0  0 10 10  1  0  0  1 10  1  6 10  6  1  6  6  0  6  1
  0  0 10 10  0  1 10  6  1  6  0  1  0 10  6  0 1

# Resultado
Cuando corremos el código del pipeline, sigue los procesos de preprocesamiento, a continuación del de análisis de datos. Con ello observamos en pantalla el dataset de prueba, y la predicción del número de hilos que se prevee utilizar según la implementación. También observamos la precisión de nuestro modelo con una precisión que ronda entre 80 y 85%